In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Ovarian_Cancer"
cohort = "GSE135820"

# Input paths
in_trait_dir = "../../input/GEO/Ovarian_Cancer"
in_cohort_dir = "../../input/GEO/Ovarian_Cancer/GSE135820"

# Output paths
out_data_file = "../../output/preprocess/Ovarian_Cancer/GSE135820.csv"
out_gene_data_file = "../../output/preprocess/Ovarian_Cancer/gene_data/GSE135820.csv"
out_clinical_data_file = "../../output/preprocess/Ovarian_Cancer/clinical_data/GSE135820.csv"
json_path = "../../output/preprocess/Ovarian_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE135820_family.soft.gz', 'GSE135820_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Ovarian_Cancer/GSE135820/GSE135820_family.soft.gz
Matrix file: ../../input/GEO/Ovarian_Cancer/GSE135820/GSE135820_series_matrix.txt.gz


Background Information:
!Series_title	"Development and validation of PrOTYPE (Predictor of high-grade-serous Ovarian carcinoma molecular subTYPE)"
!Series_summary	"Gene expression-based molecular subtypes of high grade serous tubo-ovarian cancer (HGSOC) are distinguished by differential immune and stromal infiltration and may provide opportunities for the development of targeted therapies. Integration of molecular subtypes into clinical trials has been hindered by inconsistent subtyping methodology. Adopting two independent approaches, we derived and internally validated algorithms for molecular subtype prediction from gene-expression array data in 1650 tumors. We applied resulting models to assign labels to 3829 HGSOCs from the Ovarian Tumor Tissue Analysis (OTTA) consortium evaluated on NanoString. Using the labeled NanoString data, we developed, confirmed, and validated a clinical-grade test and prediction tool. We also used the OTTA dataset to evaluate associations between molecula

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Part 1: Check if gene expression data is available
# This is a high-grade serous ovarian cancer study with gene expression data
is_gene_available = True

# Part 2: Identify available clinical features and create conversion functions

# 2.1 Trait Availability
# Looking at the sample characteristics dictionary, key 0 contains diagnosis information
trait_row = 0

# Age Availability - Key 3 contains age information
age_row = 3

# Gender Availability - No gender information available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> int:
    """
    Convert trait value to binary format.
    HGSOC (High Grade Serous Ovarian Cancer) = 1
    non-HGSOC = 0
    """
    if value is None:
        return None
    
    # Extract value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'HGSOC' in value and 'non-HGSOC' not in value:
        return 1
    elif 'non-HGSOC' in value:
        return 0
    return None

def convert_age(value: str) -> float:
    """
    Convert age value to continuous format.
    """
    if value is None:
        return None
    
    # Extract value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> int:
    """
    Placeholder function since gender data is not available.
    """
    return None

# Part 3: Save Metadata (Initial Filtering)
# Determine trait availability
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Part 4: Clinical Feature Extraction (if trait_row is not None)
if trait_row is not None:
    # Extract clinical features using the geo_select_clinical_features function
    clinical_selected = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical dataframe
    preview = preview_df(clinical_selected)
    print("Clinical data preview:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_selected.to_csv(out_clinical_data_file, index=False)


Clinical data preview:
{'GSM4030662': [1.0, 56.0], 'GSM4030663': [1.0, 58.0], 'GSM4030664': [1.0, 43.0], 'GSM4030665': [1.0, 61.0], 'GSM4030666': [1.0, 75.0], 'GSM4030667': [1.0, 60.0], 'GSM4030668': [1.0, 49.0], 'GSM4030669': [1.0, 56.0], 'GSM4030670': [1.0, 75.0], 'GSM4030671': [1.0, 64.0], 'GSM4030672': [1.0, 58.0], 'GSM4030673': [1.0, 64.0], 'GSM4030674': [1.0, 58.0], 'GSM4030675': [1.0, 50.0], 'GSM4030676': [1.0, 45.0], 'GSM4030677': [1.0, 57.0], 'GSM4030678': [1.0, 59.0], 'GSM4030679': [1.0, 60.0], 'GSM4030680': [1.0, 68.0], 'GSM4030681': [1.0, 66.0], 'GSM4030682': [1.0, 52.0], 'GSM4030683': [1.0, 60.0], 'GSM4030684': [1.0, 49.0], 'GSM4030685': [1.0, 53.0], 'GSM4030686': [1.0, 79.0], 'GSM4030687': [1.0, 45.0], 'GSM4030688': [1.0, 51.0], 'GSM4030689': [1.0, 66.0], 'GSM4030690': [1.0, 58.0], 'GSM4030691': [1.0, 51.0], 'GSM4030692': [1.0, 63.0], 'GSM4030693': [1.0, 60.0], 'GSM4030694': [1.0, 56.0], 'GSM4030695': [1.0, 55.0], 'GSM4030696': [1.0, 45.0], 'GSM4030697': [1.0, 65.0], 'GSM

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 513
First 20 gene/probe identifiers:
Index(['AJ294735.1:15', 'ENST00000390559.1:246', 'NM_000038.3:6850',
       'NM_000051.3:1561', 'NM_000055.2:1445', 'NM_000059.3:115',
       'NM_000075.2:1055', 'NM_000077.4:673', 'NM_000089.3:2635',
       'NM_000090.3:180', 'NM_000093.3:6345', 'NM_000125.2:1595',
       'NM_000138.3:6420', 'NM_000149.3:340', 'NM_000166.5:165',
       'NM_000181.3:1899', 'NM_000194.1:240', 'NM_000222.1:5',
       'NM_000245.2:405', 'NM_000248.2:624'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be RefSeq transcript IDs with position information
# Format is typically: NM_XXXXXX.X:YYYY where XXXXXX is the accession number,
# X is the version, and YYYY is likely a position within the transcript.
# These are not standard human gene symbols and would need to be mapped to gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['NM_001086.2:90', 'NM_005502.3:4936', 'NM_000927.3:3910', 'NM_000392.3:3150', 'NM_001144070.1:460'], 'ORF': ['AADAC', 'ABCA1', 'ABCB1', 'ABCC2', 'ABCC3'], 'GB_ACC': ['NM_001086.2', 'NM_005502.3', 'NM_000927.3', 'NM_000392.3', 'NM_001144070.1'], 'Target.Region': ['91-190', '4937-5036', '3911-4010', '3151-3250', '461-560'], 'SEQUENCE': ['ATGGGAAGAAAATCGCTGTACCTTCTGATTGTGGGGATCCTCATAGCATATTATATTTATACGCCTCTCCCAGATAACGTTGAGGAGCCATGGAGAATGA', 'GACGTATGTGCAGATCATAGCCAAAAGCTTAAAGAACAAGATCTGGGTGAATGAGTTTAGGTATGGCGGCTTTTCCCTGGGTGTCAGTAATACTCAAGCA', 'TATAGCACTAAAGTAGGAGACAAAGGAACTCAGCTCTCTGGTGGCCAGAAACAACGCATTGCCATAGCTCGTGCCCTTGTTAGACAGCCTCATATTTTGC', 'CAGTGACTCTAAAATCTTCAATAGCACCGACTATCCAGCATCTCAGAGGGACATGAGAGTTGGAGTCTACGGAGCTCTGGGATTAGCCCAAGGTATATTT', 'GCTGCAGGGCGTACAGTCTTCGGGGGTCCTCATTATCTTCTGGTTCCTGTGTGTGGTCTGCGCCATCGTCCCATTCCGCTCCAAGATCCTTTTAGCCAAG'], 'Customer.Identifier': ['AADAC', 'ABCA1', 'ABCB1', 'ABCC2', 'ABCC3'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# Examine the gene annotation dataframe to identify the appropriate columns for mapping
print("Gene annotation columns:", gene_annotation.columns.tolist())

# From the preview, we can see that:
# - 'ID' column in gene_annotation contains identifiers similar to those in gene expression data
# - 'ORF' column appears to contain gene symbols

# 1. Decide which columns to use for mapping
probe_id_column = 'ID'  # Column with probe/transcript IDs matching gene expression data
gene_symbol_column = 'ORF'  # Column with gene symbols

print(f"Using '{probe_id_column}' for probe IDs and '{gene_symbol_column}' for gene symbols")

# 2. Get gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, probe_id_column, gene_symbol_column)
print("Gene mapping dataframe shape:", mapping_df.shape)
print("Sample of gene mapping data:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print("\nGene expression data after mapping:")
print("Number of genes:", len(gene_data))
print("Number of samples:", len(gene_data.columns))
print("First few genes:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene annotation columns: ['ID', 'ORF', 'GB_ACC', 'Target.Region', 'SEQUENCE', 'Customer.Identifier', 'SPOT_ID']
Using 'ID' for probe IDs and 'ORF' for gene symbols
Gene mapping dataframe shape: (2096091, 2)
Sample of gene mapping data:
                   ID   Gene
0      NM_001086.2:90  AADAC
1    NM_005502.3:4936  ABCA1
2    NM_000927.3:3910  ABCB1
3    NM_000392.3:3150  ABCC2
4  NM_001144070.1:460  ABCC3



Gene expression data after mapping:
Number of genes: 509
Number of samples: 4077
First few genes:
Index(['AADAC', 'ABCA1', 'ABCB1', 'ABCC2', 'ABCC3', 'ABCC4', 'ABCC5', 'ABCE1',
       'ADAM12', 'ADAMDEC1'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Ovarian_Cancer/gene_data/GSE135820.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using the NCBI Gene database synonym information
print("Normalizing gene symbols using NCBI synonym information...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Number of genes before normalization: {len(gene_data)}")
print(f"Number of genes after normalization: {len(normalized_gene_data)}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Since we determined in step 2 that there's no usable trait variation 
# (all samples are cancer cases with no controls), we'll create a clinical dataframe
# but note this limitation

# Create a clinical dataframe with the trait (Ovarian_Cancer)
sample_ids = normalized_gene_data.columns.tolist()
print(f"Sample IDs from gene data: {len(sample_ids)} samples")

# Create clinical dataframe, but note that all samples have the same trait value
clinical_df = pd.DataFrame(index=[trait], columns=sample_ids)
clinical_df.loc[trait] = 1  # All samples are ovarian cancer tumors

print(f"Clinical data shape: {clinical_df.shape}")

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
print("Handling missing values...")
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait and demographic features are biased
print("Checking for bias in features...")
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 6. Validate the dataset and save cohort information
note = "Dataset contains expression data for ovarian cancer tumors. All samples are tumor samples with no controls, so trait bias is expected and the dataset is not suitable for case-control analysis."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True, 
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note=note
)

# 7. Save the linked data if it's usable (though we expect it won't be due to trait bias)
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed due to trait bias. Final linked data not saved.")

Normalizing gene symbols using NCBI synonym information...
Number of genes before normalization: 509
Number of genes after normalization: 509


Normalized gene expression data saved to ../../output/preprocess/Ovarian_Cancer/gene_data/GSE135820.csv
Sample IDs from gene data: 4077 samples
Clinical data shape: (1, 4077)
Clinical data saved to ../../output/preprocess/Ovarian_Cancer/clinical_data/GSE135820.csv
Shape of linked data: (4077, 510)
Handling missing values...


Shape of linked data after handling missing values: (4077, 510)
Checking for bias in features...
Quartiles for 'Ovarian_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Ovarian_Cancer' in this dataset is severely biased.

Dataset validation failed due to trait bias. Final linked data not saved.


/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())
